# RPGRIP1
Date from [Beryozkin A, et al. (2021) Retinal Degeneration Associated With RPGRIP1: A Review of Natural History, Mutation Spectrum, and Genotype-Phenotype Correlation in 228 Patients. Front Cell Dev Biol. 2021 Oct 14;9:746781. doi: 10.3389/fcell.2021.746781. PMID: 34722527; PMCID: PMC8551679.]().
RPGRIP1 encodes a ciliary protein expressed in the photoreceptor connecting cilium. Mutations in this gene cause ∼5% of Leber congenital amaurosis (LCA) worldwide, but are also associated with cone-rod dystrophy (CRD) and retinitis pigmentosa (RP) phenotypes. Our purpose was to clinically characterize RPGRIP1 patients from our cohort, collect clinical data of additional RPGRIP1 patients reported previously in the literature, identify common clinical features, and seek genotype-phenotype correlations.

In [1]:
import pandas as pd
from collections import defaultdict
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.66


In [2]:
PMID = "PMID:34722527"
title = "Retinal Degeneration Associated With RPGRIP1: A Review of Natural History, Mutation Spectrum, and Genotype-Phenotype Correlation in 228 Patients"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-03-06


In [3]:
df = pd.read_excel('input/Table_2_Beryozkin.xlsx')
df = df.astype(str)

In [4]:
df.head(2)

,# of family or patient,Article title,First author,First mutation (C.),First mutation (P.),Second mutation (C.),Second mutation (P.),Diagnosis,Intelectual disability/ Neurodevelopmental delay,First sighns and symptoms,...,Nyctalopia (night blidness),ODR (oculodigital reflex),Refraction/RNS (cycloplaegic retinoscopy spherical equivalent),VA,age of VA,VF,age of VF,rod,cone,age of ERG
0,P030,Targeted next generation sequencing identified novel mutations in RPGRIP1 associated with both retinitis pigmentosa and Leber's congenital amaurosis in unrelated Chinese patients,Hui Huang,Exon 1-22 deletion,nan,Exon 1-22 deletion,nan,LCA,no,0.0,...,nan,nan,nan,0.001,36.0,nan,nan,not detectable,not detectable,36.0
1,41,Clinical and Genetic Evaluation of a Cohort of Pediatric Patients with Severe Inherited Retinal Dystrophies,Valentina Di Iorio,c.86-3T>G,Splicing,c.2225_2226del,p.(G742fs),LCA,no,0.1,...,nan,nan,nan,0.05,8.0,nan,nan,0,0,8.0


In [5]:
# create a column with a unique patient id
from collections import defaultdict
# For each publication, count from 1 to k
# The original supplemental file does not always provide an id, but we need one
patient_id_d = defaultdict(int)

def unique_id(row:pd.Series):
    number = str(row["# of family or patient"])
    art_title = str(row["Article title"])
    art_title = art_title[:15]
    first_author = str(row["First author"])
    first_author = first_author.replace(" ", "_").replace(",", "_").replace(".", "_")
    number = number.replace(" ", "")
    title = art_title.replace(" ", "")
    patient_id_part = f"{first_author}_{title}"
    first_author = first_author.replace(" ", "")
    if number == "nan" or len(number) < 2:
        patient_id_d[patient_id_part] += 1
        p_id = patient_id_d.get(patient_id_part)
        p_id = f"individual_{p_id}"
    else:
        patient_id_d[patient_id_part] += 1
        p_id = patient_id_d.get(patient_id_part) 
        p_id = f"{number}_{p_id}"
    return f"individual_{p_id}_{patient_id_part}"

df["individual_id"] = df.apply(unique_id, axis=1)
#df["individual_id"]

In [6]:
column_mapper_list = list()

In [7]:
idMapper = SimpleColumnMapper(column_name='Intelectual disability/ Neurodevelopmental delay',
                              hpo_id="HP:0001263", hpo_label="Global developmental delay", observed="yes", excluded="no")
column_mapper_list.append(idMapper)
idMapper.preview_column(df)

,mapping,count
0,"original value: ""no"" -> HP: Global developmental delay (HP:0001263) (excluded)",217
1,"original value: ""yes"" -> HP: Global developmental delay (HP:0001263) (observed)",12


In [8]:
nystagmusMapper = SimpleColumnMapper(column_name='Nystagmus/wandering/no fixation',
                                     hpo_id="HP:0000639", hpo_label="Nystagmus", observed="1.", excluded="no")
column_mapper_list.append(nystagmusMapper)
nystagmusMapper.preview_column(df)

,mapping,count
0,"original value: ""1.0"" -> HP: Nystagmus (HP:0000639) (not measured)",103
1,"original value: ""nan"" -> HP: Nystagmus (HP:0000639) (not measured)",126


In [9]:
photophobiaMapper = SimpleColumnMapper(column_name='Photophobia',
                                       hpo_id="HP:0000613", hpo_label="Photophobia", observed="1.", excluded="n")
column_mapper_list.append(photophobiaMapper)
photophobiaMapper.preview_column(df)

,mapping,count
0,"original value: ""nan"" -> HP: Photophobia (HP:0000613) (not measured)",195
1,"original value: ""1.0"" -> HP: Photophobia (HP:0000613) (not measured)",34


In [10]:
# Nyctalopia (night blidness)
nyctalopiaMapper = SimpleColumnMapper(column_name='Nyctalopia (night blidness)',
                                      hpo_id="HP:0000662", hpo_label="Nyctalopia", observed="1.", excluded="n")
column_mapper_list.append(nyctalopiaMapper)
nyctalopiaMapper.preview_column(df)

,mapping,count
0,"original value: ""nan"" -> HP: Nyctalopia (HP:0000662) (not measured)",218
1,"original value: ""1.0"" -> HP: Nyctalopia (HP:0000662) (not measured)",11


In [11]:
# Eye poking HP:0001483
odrMapper = SimpleColumnMapper(column_name='ODR (oculodigital reflex)',
                               hpo_id="HP:0001483", hpo_label="Eye poking", observed=["yes","1"], excluded="no")
column_mapper_list.append(odrMapper)
odrMapper.preview_column(df)

,mapping,count
0,"original value: ""nan"" -> HP: Eye poking (HP:0001483) (not measured)",190
1,"original value: ""yes"" -> HP: Eye poking (HP:0001483) (observed)",27
2,"original value: ""1"" -> HP: Eye poking (HP:0001483) (observed)",1
3,"original value: ""no"" -> HP: Eye poking (HP:0001483) (excluded)",11


In [12]:
#results = OptionColumnMapper.autoformat(df, concept_recognizer=hpo_cr)
#print(results)

In [13]:
refraction_d = {
 '2.5': 'Moderate hypermetropia',
 '6.5': 'High hypermetropia',
 '10': 'High hypermetropia',
 '2.25': 'Moderate hypermetropia',
 '1.5': 'Mild hypermetropia',
 '3.75': 'Moderate hypermetropia',
 '2.75': 'Moderate hypermetropia',
 '1.25': 'Mild hypermetropia',
 'hyperopia': 'Hypermetropia',
 '4.5': 'Moderate hypermetropia',
 '10.25': 'High hypermetropia',
 '3.5': 'Moderate hypermetropia',
 '2.6': 'Moderate hypermetropia',
 '6.9': 'High hypermetropia',
 'high myopia': 'High myopia',
 '3.25': 'Moderate hypermetropia',
 '5.5': 'High hypermetropia'}
refractionMapper = OptionColumnMapper(column_name='Refraction/RNS (cycloplaegic retinoscopy spherical equivalent)',
                                      concept_recognizer=hpo_cr, option_d=refraction_d)
column_mapper_list.append(refractionMapper)
refractionMapper.preview_column(df)

,mapping,count
0,Moderate hypermetropia (HP:0031729) (observed),12
1,High hypermetropia (HP:0008499) (observed),9
2,Mild hypermetropia (HP:0031728) (observed),2
3,Hypermetropia (HP:0000540) (observed),1
4,High myopia (HP:0011003) (observed),4


In [14]:
va_d = {'0.001': 'Very low visual acuity',
 '0.05': 'Severely reduced visual acuity',
 '0.03': 'Very low visual acuity',
 '0.0': 'Very low visual acuity',
 '0.1': 'Moderately reduced visual acuity',
 '0.01': 'Very low visual acuity',
 '0.0001': 'Very low visual acuity',
 '0.025': 'Very low visual acuity',
 '0.15': 'Moderately reduced visual acuity',
 '0.15037593984962405': 'Moderately reduced visual acuity',
 '0.25': 'Moderately reduced visual acuity',
 '0.41': 'Moderately reduced visual acuity',
 '0.13333333333333333': 'Moderately reduced visual acuity',
 '0.06': 'Severely reduced visual acuity',
 '0.2': 'Moderately reduced visual acuity',
 '0.5': 'Mildly reduced visual acuity',
 '0.41500000000000004': 'Moderately reduced visual acuity',
 '0.4': 'Mildly reduced visual acuity',
 '0.09': 'Severely reduced visual acuity',
 '0.02': 'Very low visual acuity',
 '0.07500000000000001': 'Very low visual acuity'}
vaMapper = OptionColumnMapper(column_name='VA',concept_recognizer=hpo_cr, option_d=va_d)
column_mapper_list.append(vaMapper)
vaMapper.preview_column(df)

,mapping,count
0,Very low visual acuity (HP:0032122) (observed),89
1,Severely reduced visual acuity (HP:0001141) (observed),6
2,Moderately reduced visual acuity (HP:0030515) (observed),24
3,Mildly reduced visual acuity (HP:0032037) (observed),3


In [15]:
# A normal visual field is an island of vision measuring 90 degrees temporally to central Fixation, 
# 50 degrees superiorly and nasally, and 60 degrees inferiorly. 
vf_d = {
 '30': 'Constriction of peripheral visual field',
 '12.5': 'Constriction of peripheral visual field',
 '40': 'Constriction of peripheral visual field',
 '10': 'Constriction of peripheral visual field',
 '15': 'Constriction of peripheral visual field',
 '20': 'Constriction of peripheral visual field',
 'severly constricted': 'Constriction of peripheral visual field'}
vfMapper = OptionColumnMapper(column_name='VF',concept_recognizer=hpo_cr, option_d=vf_d)
column_mapper_list.append(vfMapper)
vfMapper.preview_column(df)

,mapping,count
0,Constriction of peripheral visual field (HP:0001133) (observed),10


In [16]:
# We will not attempt to map ERG findings in more detail because very scant information is provided
rod_d = {'not detectable': 'Abnormal electroretinogram',
 'no detectable responses': 'Abnormal electroretinogram',
 'flat': 'Abnormal electroretinogram',
 'subnormal': 'Abnormal electroretinogram',
 'extinguished': 'Abnormal electroretinogram',
 'non-recordable': 'Abnormal electroretinogram',
 'Reduced and delayed': 'Abnormal electroretinogram',
 'reduced': 'Abnormal electroretinogram',
 'not detected': 'Abnormal electroretinogram'}
excluded = { 'norm': 'Abnormal electroretinogram',}
rodMapper = OptionColumnMapper(column_name='rod',concept_recognizer=hpo_cr, option_d=rod_d, excluded_d=excluded)
column_mapper_list.append(rodMapper)
rodMapper.preview_column(df)

,mapping,count
0,Abnormal electroretinogram (HP:0000512) (observed),27
1,Abnormal electroretinogram (HP:0000512) (excluded),1


In [17]:
cone_d = {'not detectable': 'Abnormal electroretinogram',
 'no detectable responses': 'Abnormal electroretinogram',
 'flat': 'Abnormal electroretinogram',
 'extinguished': 'Abnormal electroretinogram',
 'non-recordable': 'Abnormal electroretinogram',
 'severly reduced': 'Abnormal electroretinogram',
 'not detected': 'Abnormal electroretinogram'}
excluded = { 'norm': 'Abnormal electroretinogram',}
coneMapper = OptionColumnMapper(column_name='cone',concept_recognizer=hpo_cr, option_d=cone_d, excluded_d=excluded)
column_mapper_list.append(coneMapper)
coneMapper.preview_column(df)

,mapping,count
0,Abnormal electroretinogram (HP:0000512) (observed),28
1,Abnormal electroretinogram (HP:0000512) (excluded),1


# Variants¶

Note that we corrected some HGVS, e.g. c.3618-5_c.3618-1del should be c.3618-1_3621del

In [18]:
RPGRIP1_transcript = "NM_020366.4"
RPGRIP1_id = "HGNC:13436"
vman = VariantManager(df=df, individual_column_name="individual_id", allele_1_column_name='First mutation (C.)',
                     allele_2_column_name="Second mutation (C.)", gene_id=RPGRIP1_id, gene_symbol="RPGRIP1", transcript=RPGRIP1_transcript)

In [19]:
vman.to_summary()

,status,count,alleles
0,mapped,119,"c.1116delA, c.420delG, c.2237G>A, c.535delG, c.2417C>T, c.2020C>T, c.1624delG, c.2576_2577insTT, c.2644C>T, c.3330T>A, c.2710+372_2895+76del, c.1763-2A>G, c.2012G>A, c.800G>A, c.2314C>T, c.2398G>A, c.2555G>A, c.2566_2567insTT, c.2941C>T, c.1111C>T, c.3565C>T, c.1525C>T, c.3560_3566del, c.367C>T, c.2890delT, c.1892A>C, c.3565_3571delCGAAGGC, c.1687C>T, c.2009_2010insG, c.2465_2468dup, c.2935C>T, c.1611G>A, c.534delG, c.1892A>T, c.2796dup, c.194G>A, c.2302C>T, c.931delA, c.1084_1087del, c.2440C>T, c.3388G>C, c.1083_1084insGA, c.1216del, c.86-3T>G, c.2367+23del, c.961_962insCCCT, c.3618-1_3621del, c.3663_3666del, c.2608_2609insA, c.2435G>A, c.1234C>T, c.767C>G, c.2261delT, c.3793_3794insGAAA, c.2718dup, c.242T>A, c.2656C>T, c.2554C>T, c.2209_2215+18del, c.1107delA, c.2795_2796insT, c.3100_3238del139, c.2021C>A, c.2592T>G, c.2758C>T, c.2480G>T, c.1306G>T, c.490+1G>T, c.630del, c.800+1G>A, c.2236G>A, c.1468-2A>G , c.2468A>G, c.1303A>T, c.1145T>A, c.1210G>T, c.1445T>A, c.3609delT, c.2058dupA, c.1467+1G>T, c.2889delT, c.511delT, c.358C>T, c.2710+1G>A, c.3629_3630insG, c.3560_3566delAAGGCCG, c.2225_2226del, c.895_896del, c.154C>T, c.1447C>T, c.2356C>T, c.930+1G>A, c.1180C>T, c.3632T>A, c.2079C>G, c.711delA, c.1148_1151del, c.587+1G>C, c.3749-1G>T, c.1468-2A>G, c.2295_2296delinsAA, c.3617+1G>A, c.1505_1506insTGTC, c.2759_2760insT, c.801-25_843del, c.1615_1624del10, c.1225C>T, c.2662C>T, c.3835_3837delGAG, c.510delC, c.2668C>T, c.2249A>G, c.2974delA, c.799C>T, c.1134_1136delTGA, c.3358A>G, c.3620T>G, c.2710+374_2895+78del, c.3748+1C>A"
1,unmapped,6,"Exon19 del, Exon 17–19 deletion, Exon 10–18 deletion, Exon 2 duplication, Exon 1–2 dup, Exon 1-22 deletion"


In [20]:
svars_del = {  "Exon 10–18 deletion" , "Exon19 del", "Exon 1-22 deletion", "Exon 17–19 deletion"}
svar_dup = {"Exon 1–2 dup", "Exon 2 duplication"}
vman.code_as_chromosomal_deletion(svars_del)
vman.code_as_chromosomal_duplication(svar_dup)
vman.to_summary()

,status,count,alleles
0,mapped,125,"c.1116delA, c.420delG, c.2237G>A, c.535delG, c.2417C>T, c.2020C>T, c.1624delG, c.2576_2577insTT, c.2644C>T, c.3330T>A, c.2710+372_2895+76del, c.1763-2A>G, c.2012G>A, c.800G>A, c.2314C>T, c.2398G>A, c.2555G>A, c.2566_2567insTT, c.2941C>T, c.1111C>T, c.3565C>T, c.1525C>T, c.3560_3566del, c.367C>T, c.2890delT, c.1892A>C, c.3565_3571delCGAAGGC, c.1687C>T, c.2009_2010insG, c.2465_2468dup, c.2935C>T, c.1611G>A, c.534delG, c.1892A>T, c.2796dup, c.194G>A, c.2302C>T, c.931delA, c.1084_1087del, c.2440C>T, c.3388G>C, c.1083_1084insGA, c.1216del, c.86-3T>G, c.2367+23del, c.961_962insCCCT, c.3618-1_3621del, c.3663_3666del, c.2608_2609insA, c.2435G>A, c.1234C>T, c.767C>G, c.2261delT, c.3793_3794insGAAA, c.2718dup, c.242T>A, c.2656C>T, c.2554C>T, c.2209_2215+18del, c.1107delA, c.2795_2796insT, c.3100_3238del139, c.2021C>A, c.2592T>G, c.2758C>T, c.2480G>T, c.1306G>T, c.490+1G>T, c.630del, c.800+1G>A, c.2236G>A, c.1468-2A>G , c.2468A>G, c.1303A>T, c.1145T>A, c.1210G>T, c.1445T>A, c.3609delT, c.2058dupA, c.1467+1G>T, c.2889delT, c.511delT, c.358C>T, c.2710+1G>A, c.3629_3630insG, c.3560_3566delAAGGCCG, c.2225_2226del, c.895_896del, c.154C>T, c.1447C>T, c.2356C>T, c.930+1G>A, c.1180C>T, c.3632T>A, c.2079C>G, c.711delA, c.1148_1151del, c.587+1G>C, c.3749-1G>T, c.1468-2A>G, c.2295_2296delinsAA, c.3617+1G>A, c.1505_1506insTGTC, c.2759_2760insT, c.801-25_843del, c.1615_1624del10, c.1225C>T, c.2662C>T, c.3835_3837delGAG, c.510delC, c.2668C>T, c.2249A>G, c.2974delA, c.799C>T, c.1134_1136delTGA, c.3358A>G, c.3620T>G, c.2710+374_2895+78del, c.3748+1C>A, Exon19 del, Exon 17–19 deletion, Exon 10–18 deletion, Exon 1-22 deletion, Exon 2 duplication, Exon 1–2 dup"
1,unmapped,0,


In [21]:
iso_d = {
    '0.0':"P1D", 
    '0.1':"P1M",
     '2.0':"P2Y", 
    '4.0':"P4Y", 
    '0.4':"P4M", 
    '0.5':"P6M", 
    '0.6':"P7M", 
    '1.5':"P1Y6M",
    '0.2':"P2M", 
    '6.0':"P6Y", 
    '1.0':"P1Y", 
    '0.7':"P8M", 
    '8.0':"P8Y", 
    '0.3':"P3M", 
    '9.0':"P9Y", 
    '10.0':"P10Y", 
    '12.0':"P12Y",
    '15.0':"P15Y", 
    '3.0':"P3Y", 
    '0.95':"P11M", 
    '5.0':"P5Y", 
    '13.0':"P13Y", 
    '0.8':"P8M", 
    '7.0':"P7Y",
    '11.0':"P11Y"
}

#df['First sighns and symptoms'].unique()
ageMapper = AgeColumnMapper.custom_dictionary(column_name='First sighns and symptoms',string_to_iso_d=iso_d)
#ageMapper.preview_column(df)

# Diagnosis

OMIM lists two diagnoses for the RPGRIP1. The authors list eight clinical diagnoses that we map as follows:

- [Cone-rod dystrophy 13, OMIM:608194 ](https://omim.org/entry/608194): 'RP', 'CRD', , 'eoRP', 'CRD (because cones was reduced more than rods, among 16 patients from 2 families, ages 14-16)', 'CRD ', 'RP\u2009'
- [Leber congenital amaurosis 6; OMIM:613826](https://omim.org/entry/613826): 'LCA', 'CRD/LCA'

In [22]:
crd13 = Disease(disease_id="OMIM:608194", disease_label="Cone-rod dystrophy 13")
lca6 = Disease(disease_id="OMIM:613826", disease_label="Leber congenital amaurosis 6")
disease_d = {
'RP': crd13, 
    'CRD': crd13,
    'eoRP': crd13, 
    'CRD (because cones was reduced more than rods, among 16 patients from 2 families, ages 14-16)': crd13,
    'CRD ': crd13,
    'RP\u2009': crd13,
    'LCA':lca6, 
    'CRD/LCA': lca6
}

In [23]:
var_d = vman.get_variant_d()
patient_id_to_var_list_d = defaultdict(list)
patient_id_to_disease_d = {}
for _, row in df.iterrows():
    iid = row['individual_id']
    v1 = row['First mutation (C.)']
    v2 = row['Second mutation (C.)']
    var1 = var_d.get(v1)
    if var1 is None:
        raise ValueError(f"Could not retrieve variant for {v1}")
    var2 = var_d.get(v2)
    if var2 is None:
        raise ValueError(f"Could not retrieve variant for {v2}")
    patient_id_to_var_list_d[iid].append(var1)
    patient_id_to_var_list_d[iid].append(var2)
    dx = row['Diagnosis']
    disease = disease_d.get(dx)
    if disease is None:
        raise ValueError(f"Could not retrieve disease for {dx}")
    patient_id_to_disease_d[iid] = disease

In [24]:
encoder = CohortEncoder(df=df,
                       hpo_cr=hpo_cr,
                        column_mapper_list=column_mapper_list,
                        individual_column_name='individual_id',
                        metadata=metadata,
                        age_at_last_encounter_mapper=AgeColumnMapper.not_provided(),
                        sexmapper=SexColumnMapper.not_provided()
                    )
encoder.set_disease_dictionary(patient_id_to_disease_d)
individuals = encoder.get_individuals()
for ii in individuals:
    i_id = ii.id
    var_list = patient_id_to_var_list_d.get(i_id)
    if var_list is None or len(var_list) != 2:
        raise ValueError(f"Malformed variant list for {i_id}: {var_list}")
    else:
        var1 = var_list[0]
        var2 = var_list[1]
        if var1 == var2:
            var1.set_homozygous()
            ii.add_variant(var1)
        else:
            var1.set_heterozygous()
            var2.set_heterozygous()
            ii.add_variant(var1)
            ii.add_variant(var2)

In [25]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,877


In [26]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
individual_P030_1_Hui_Huang_Targetednextg (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),Exon 1-22 deletion: chromosomal_deletion (SO:1000029),Very low visual acuity (HP:0032122); Abnormal electroretinogram (HP:0000512); excluded: Global developmental delay (HP:0001263)
individual_41_1_Valentina_Di_Iorio_ClinicalandGe (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.86-3T>G (heterozygous) NM_020366.4:c.2227_2228del (heterozygous),Severely reduced visual acuity (HP:0001141); excluded: Global developmental delay (HP:0001263)
individual_arRP-F083_1_Leen_Abu-Safieh_Autozygome-guid (UNKNOWN; n/a),Cone-rod dystrophy 13 (OMIM:608194),NM_020366.4:c.154C>T (homozygous),excluded: Global developmental delay (HP:0001263)
individual_individual_1_Juan_C__Zenteno_Extensivegenic (UNKNOWN; n/a),Cone-rod dystrophy 13 (OMIM:608194),NM_020366.4:c.154C>T (homozygous),excluded: Global developmental delay (HP:0001263)
individual_P024_2_Hui_Huang_Targetednextg (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.154C>T (heterozygous) NM_020366.4:c.2020C>T (heterozygous),Very low visual acuity (HP:0032122); Abnormal electroretinogram (HP:0000512); excluded: Global developmental delay (HP:0001263)
individual_individual_1_Cathrine_Jespersgaard_Moleculargenet (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.194G>A (homozygous),Global developmental delay (HP:0001263)
individual_individual_1_Jana_Zernant_GenotypingMicr (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.194G>A (homozygous),excluded: Global developmental delay (HP:0001263)
individual_individual_1_Lin_Li_DetectionofVa (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.242T>A (homozygous),excluded: Global developmental delay (HP:0001263)
individual_QT587_2_Lin_Li_DetectionofVa (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.358C>T (homozygous),excluded: Global developmental delay (HP:0001263)
individual_QT491_3_Lin_Li_DetectionofVa (UNKNOWN; n/a),Leber congenital amaurosis 6 (OMIM:613826),NM_020366.4:c.358C>T (homozygous),excluded: Global developmental delay (HP:0001263)


In [27]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,
                                             metadata=metadata)

We output 229 GA4GH phenopackets to the directory phenopackets


In [28]:
# pxf validate --hpo hp.json  *.json
# no errors